# Get music features from Spotipy API for Billboard Top 100 songs

In [1]:
#import libraries
import pandas as pd
import numpy as np

In [2]:
file_path="https://raw.githubusercontent.com/hanna-freuden/musiclyrics-1/master/billboard_lyrics_1964-2015.csv"
df = pd.read_csv(file_path,delimiter=',', encoding='latin-1')

In [3]:
df.columns

Index(['Rank', 'Song', 'Artist', 'Year', 'Lyrics', 'Source'], dtype='object')

In [4]:
#get list of unique artists for spotipy API
artists=set(df['Artist'].tolist())

In [5]:
df.tail()

,Rank,Song,Artist,Year,Lyrics,Source
5095,96,el perdon,nicky jam and enrique iglesias,2015,enrique iglesias dime si es verdad me dijeron ...,3.0
5096,97,she knows,neyo featuring juicy j,2015,NaN,NaN
5097,98,night changes,one direction,2015,going out tonight changes into something red ...,1.0
5098,99,back to back,drake,2015,oh man oh man oh man not againyeah i learned ...,1.0
5099,100,how deep is your love,calvin harris and disciples,2015,i want you to breathe me in let me be your ai...,1.0


In [6]:
df.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [7]:
#spotify api access
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
from tqdm import notebook
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '12b175848fa041edb2b6981d157bb690'
client_secret = '2eb73ed188c94048a8bb392f0df37403'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [ ]:
#get genre of all artists
genres={}

for artist in notebook.tqdm(artists):
    genres[artist]=sp.artist(sp.search(artist)['tracks']['items'][0]['artists'][0]['uri']).get('genres')

In [6]:
#get all songs from these artists using spotipy api


def artist_tracks(artists):
    
    '''
    Takes a list of artist names, iterates through their Spotify albums, checks for 
    duplicate albums, then appends all the tracks in those albums to a list of lists
    '''
    
    # Each list in this list will be a track and its features
    tracks = []
    
    
    
    for artist in notebook.tqdm(artists):
        try:    
            # Get the artist URI (a unique ID)
            artist_uri = sp.search(artist)['tracks']['items'][0]['artists'][0]['uri']

            # Spotify has a lot of duplicate albums, but we'll cross-reference them with this list to avoid extra loops
            album_checker = []

            # The starting point of our loop of albums for those artists with more than 50
            n = 0

            # Note the album_type = 'album'. This discounts singles, compilations and collaborations
            while len(sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']) > 0:

                # Avoid overloading Spotify with requests by assigning the list of album dictionaries to a variable
                dict_list = sp.artist_albums(artist_uri, album_type = 'album', limit=50, offset = n)['items']

                for i, album in notebook.tqdm(enumerate(dict_list)):

                    # Add the featured artists for the album in question to the checklist
                    check_this_album = [j['name'] for j in dict_list[i]['artists']]
                    # And the album name
                    check_this_album.append(dict_list[i]['name'])
                    # And its date
                    check_this_album.append(dict_list[i]['release_date'])

                    # Only continue looping if that album isn't in the checklist
                    if check_this_album not in album_checker:

                        # Add this album to the checker
                        album_checker.append(check_this_album)
                        # For every song on the album, get its descriptors and features in a list and add to the tracklist
                        tracks.extend([[artist, album['name'], album['uri'], song['name'],

                          album['release_date']] + list(sp.audio_features(song['uri'])[0].values()) 
                                       for song in sp.album_tracks(album['uri'])['items']])

                # Go through the next 50 albums (otherwise we'll get an infinite while loop)
                n += 50
            
        except:
                pass

    return tracks

In [8]:
def artist_all_tracks(artists):
    
    '''
    Takes a list of artist names, iterates through their Spotify albums (including
    singles, compilations and collaborations), checks for duplicate albums, then
    appends all the tracks in those albums to a list of lists
    '''
    
    # Each list in this list will be a track and its features
    tracks = []

    
    for artist in  notebook.tqdm(artists):
        try:    
        # Get the artist URI (a unique ID)
            artist_uri = sp.search(artist)['tracks']['items'][0]['artists'][0]['uri']
           
            # Spotify has a lot of duplicate albums, but we'll cross-reference them with this list to avoid extra loops
            album_checker = []

            # The starting point of our loop of albums for those artists with more than 50
            n = 0

            # Note that here we include singles, compilations and collaborations in the albums to loop through
            while len(sp.artist_albums(artist_uri, limit=50, offset = n)['items']) > 0:

                # Avoid overloading Spotify with requests by assigning the list of album dictionaries to a variable
                dict_list = sp.artist_albums(artist_uri, limit=50, offset = n)['items']
               
                for i, album in  notebook.tqdm(enumerate(dict_list)):

                    # Add the featured artists for the album in question to the checklist
                    check_this_album = [j['name'] for j in dict_list[i]['artists']]
                    # And the album name
                    check_this_album.append(dict_list[i]['name'])
                    # And its date
                    check_this_album.append(dict_list[i]['release_date'])

                    # Only continue looping if that album isn't in the checklist
                    if check_this_album not in album_checker:

                        # Add this album to the checker
                        album_checker.append(check_this_album)
                        # For every song on the album, get its descriptors and features in a list and add to the tracklist
                        tracks.extend([[artist, album['name'], album['uri'], song['name'],

                          album['release_date']] + list(sp.audio_features(song['uri'])[0].values()) 
                                       for song in sp.album_tracks(album['uri'])['items']])

                # Go through the next 50 albums (otherwise we'll get an infinite while loop)
                n += 50
        except:
            pass
    return tracks

In [ ]:
#get tracks of all artists in artists lists
tracklist_long=artist_all_tracks(artists)

In [ ]:
#make resulting list a DataFrame and export


def df_tracks(tracklist):
    
    '''
    Takes the output of artist_tracks (i.e. a list of lists),
    puts it in a dataframe and formats it.
    '''

    df = pd.DataFrame(tracklist, columns=['artist',
     'album_name',
     'album_uri',
     'track',
     'release_date'] + list(sp.audio_features('7tr2za8SQg2CI8EDgrdtNl')[0].keys()))

    df.rename(columns={'uri':'song_uri'}, inplace=True)

    df.drop_duplicates(subset=['artist', 'track', 'release_date'], inplace=True)

    # Reorder the cols to have identifiers first, auditory features last
    cols = ['artist', 'album_name', 'album_uri', 'track', 'release_date', 'id', 'song_uri', 'track_href',
     'analysis_url', 'type', 'danceability', 'energy', 'key',  'loudness', 'mode', 'speechiness',
     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']

    df = df[cols]
    
    return df

In [ ]:
df_tracks=df_tracks(tracklist_long)

In [ ]:
#Save data frame - alternative to CSV: Pandas's to_pickle method
df_tracks.to_pickle("df_tracks_long")

In [ ]:
df_tracks.shape

In [ ]:
df_tracks['release_date']=pd.to_datetime(df_tracks['release_date'], errors='coerce')
df_tracks.dropna(inplace=True)

In [ ]:
df_tracks.head()

In [ ]:
#reduce the spotipy dataset so it only contains songs that were released 1965-2015
df_tracks['year'] = pd.DatetimeIndex(df_tracks['release_date']).year
tracks_filtered=df_tracks.loc[df_tracks['year'] >= 1964 ]
tracks_filtered=  tracks_filtered.loc[tracks_filtered['year'] <= 2015]

In [ ]:
#combine artist and song names to single sets of strings of both dataframes
#transform to lower
tracks_filtered['artist']=tracks_filtered['artist'].str.lower()
tracks_filtered['track']=tracks_filtered['track'].str.lower()
df['Artist']=df['Artist'].str.lower()
df['Song']=df['Song'].str.lower()
songs_long=list((tracks_filtered['artist']+" " +tracks_filtered['track']).unique())
songs_short=list((df['Artist']+" " +df['Song']).unique())
print(len(songs_short),len(songs_long))

In [ ]:
#delete all but letters chars from artist-song strings
import re
regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is  input string
songs_short=[regex.sub('', x) for x in songs_short]
songs_long=[regex.sub('', x) for x in songs_long]
print(len(songs_short),len(songs_long))

In [ ]:
# calculate condensed distance matrix by wrapping the Levenshtein distance function

from Levenshtein import distance
from scipy.spatial.distance import pdist, squareform
#initializing the distance matrix
strings1 = songs_short
strings2=songs_long
distances = np.zeros((len(strings1) , len(strings2) ),dtype='uint8')
for x in notebook.tqdm(range(len(strings1))):
    l = [([strings1[x]]+[i]) for i in strings2]
    for y in notebook.tqdm(range(len(strings2))):
        distances[x,y] = distance(l[y][0], l[y][1])

In [ ]:
dist=pd.DataFrame(distances)
dist

In [ ]:
dist.shape

In [ ]:
#Save data frame - alternative to CSV: Pandas's to_pickle method
dist.to_pickle("distances_long")

In [ ]:
#for each row (songs_short) get min 5 distances
#and replace cell value with respective artist-song
#transpose dist
dist_t=dist.T
min_dist={}
min_dist_values={}
for c in notebook.tqdm(dist_t.columns):
        nsmallest=dist_t[c].sort_values()[0:5]
        min_dist[c]=nsmallest.index.to_list()
        min_dist_values[c]=nsmallest.to_list()


In [ ]:
songs_long=np.asarray(songs_long)
songs_short=np.asarray(songs_short)

In [ ]:
min_dist=pd.DataFrame(min_dist).T
min_dist

In [ ]:
min_dist_values=pd.DataFrame(min_dist_values).T
min_dist_values

In [ ]:
min_dist.index=songs_short
min_dist_values.index=songs_short

In [ ]:
min_dist_songs={}
for i in range(len(songs_short)):
        min_dist_songs[i]=songs_long[min_dist[i]]

In [ ]:
df_dist=pd.DataFrame(min_dist_songs).T

In [ ]:
df_dist.index=songs_short

In [ ]:
df_dist

In [ ]:
min_5=df_dist.join(min_dist_values, lsuffix='_song', rsuffix='_dist')

In [ ]:
min_1=min_5[['0_song','0_dist']].sort_values(axis=0,by='0_dist')

In [ ]:
min_1.loc[min_1['0_dist']>10]